In [1]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import read_binary, binary_to_fp16, store_binary, generate_fp16

In [2]:
def generate_twiddles_fp16(N):
    """Generate twiddle factors (real, imag) as float16 for each stage in DIF FFT"""
    num_stages = int(np.log2(N))
    twiddles = {}

    for stage in range(1, num_stages + 1):
        step = 2 ** stage
        half = step // 2
        stage_twiddles = []

        for k in range(half):
            angle = -2 * np.pi * k / step
            real = np.float16(np.cos(angle))
            imag = np.float16(np.sin(angle))
            stage_twiddles.append([real, imag])  # Store as list of two float16 values

        twiddles[num_stages - stage] = stage_twiddles

    return twiddles

# DIF version Final

In [3]:

def fft_1(x_real, x_imag, twiddles_fp16):
    """
    Serial FFT using precomputed twiddle factors (DIF, radix-2), all in float16.

    Inputs:
        x_real, x_imag: np.float16 arrays of real and imaginary parts
        twiddles_fp16: dict[stage] = list of [real, imag] twiddles (as float16)

    Returns:
        reordered_real, reordered_imag: final output (both float16)
        tmp_value: list of intermediate outputs per stage (each is (real, imag))
    """
    x_real = np.asarray(x_real, dtype=np.float16)
    x_imag = np.asarray(x_imag, dtype=np.float16)
    N = len(x_real)
    levels = int(np.log2(N))

    # Bit-reversal permutation
    def bit_reverse(n, bits):
        rev = 0
        for _ in range(bits):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev

    reordered_real = np.zeros(N, dtype=np.float16)
    reordered_imag = np.zeros(N, dtype=np.float16)
    for i in range(N):
        j = bit_reverse(i, levels)
        reordered_real[j] = x_real[i]
        reordered_imag[j] = x_imag[i]

    tmp_value = []

    # Butterfly computation
    for stage in range(levels):  # stage: 0 to log2(N)-1
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = twiddles_fp16[levels - stage - 1]  # reverse order

        for k in range(0, N, step):
            for n in range(half_size):
                i = k + n
                j = i + half_size

                a_real = reordered_real[i]
                a_imag = reordered_imag[i]
                b_real = reordered_real[j]
                b_imag = reordered_imag[j]

                # twiddle: (wr + j*wi)
                wr, wi = stage_twiddles[n]

                # temp = twiddle * B (complex multiplication)
                temp_real = wr * b_real - wi * b_imag
                temp_imag = wr * b_imag + wi * b_real

                # A = A + temp
                reordered_real[i] = np.float16(a_real + temp_real)
                reordered_imag[i] = np.float16(a_imag + temp_imag)

                # B = A - temp
                reordered_real[j] = np.float16(a_real - temp_real)
                reordered_imag[j] = np.float16(a_imag - temp_imag)

        # Save stage output
        tmp_value.append((
            reordered_real.copy(),  # ensure copy for later reference
            reordered_imag.copy()
        ))

    # Final output
    output = []
    for i in range(N):
        output.append((reordered_real[i], reordered_imag[i]))

    return output, tmp_value

def fft(x):
    """
    Standard FFT using numpy.
    """
    x = np.asarray(x, dtype=np.complex64)
    return np.fft.fft(x)

# DIF
def fft_serial_with_twiddle_fp16(x, twiddles_fp16):
    """
    Serial FFT using precomputed twiddle factors (DIF, radix-2), all in float16.

    Inputs:
        x: list of (real, imag) tuples (float or float16)
        twiddles_fp16: dict[stage] = list of (real, imag) twiddles as float16

    Returns:
        output: list of (real, imag) float16 tuples (bit-reversed)
        stage_value: list of intermediate stage outputs (each is list of [real, imag])
    """
    x_real = np.array([np.float16(a[0]) for a in x], dtype=np.float16)
    x_imag = np.array([np.float16(a[1]) for a in x], dtype=np.float16)
    N = len(x_real)
    levels = int(np.log2(N))

    # Initialize working arrays
    reordered_real = np.copy(x_real)
    reordered_imag = np.copy(x_imag)

    stage_value = []

    # DIF Butterfly computation
    for stage in range(levels):
        tmp_value = []

        # Fix: butterfly distance starts large and halves each stage
        half_size = 2 ** (levels - stage - 1)
        step = 2 * half_size
        stage_twiddles = twiddles_fp16[stage]  # DIF uses twiddles in normal order

        for k in range(0, N, step):
            for n in range(half_size):
                i = k + n
                j = i + half_size

                a_real = reordered_real[i]
                a_imag = reordered_imag[i]
                b_real = reordered_real[j]
                b_imag = reordered_imag[j]

                # A = A + B
                t_real = np.float16(a_real + b_real)
                t_imag = np.float16(a_imag + b_imag)
                # if(k == 0):
                #     print(i, j, n, a_real, b_real, t_real)
                # B = (A - B) * twiddle
                d_real = np.float16(a_real - b_real)
                d_imag = np.float16(a_imag - b_imag)

                wr, wi = stage_twiddles[n]

                temp_real = np.float16(wr * d_real - wi * d_imag)
                temp_imag = np.float16(wr * d_imag + wi * d_real)

                reordered_real[i] = t_real
                reordered_imag[i] = t_imag
                reordered_real[j] = temp_real
                reordered_imag[j] = temp_imag

        # Save per-stage result
        for idx in range(N):
            tmp_value.append([reordered_real[idx], reordered_imag[idx]])
        stage_value.append(tmp_value.copy())

    # Bit-reversal permutation (only at the end for DIF)
    def bit_reverse(n, bits):
        rev = 0
        for _ in range(bits):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev

    final_real = np.zeros(N, dtype=np.float16)
    final_imag = np.zeros(N, dtype=np.float16)
    for i in range(N):
        j = bit_reverse(i, levels)
        final_real[j] = reordered_real[i]
        final_imag[j] = reordered_imag[i]

    # Output as list of (real, imag)
    output = [[final_real[i], final_imag[i]] for i in range(N)]

    return output, stage_value


In [26]:
# def generate_twiddle_factors_fp16_dif(N):
#     """
#     Generate twiddle factors for radix-2 DIF FFT.
#     Returns a dict: stage → list of (real, imag), already reversed for 'temp → i' structure
#     """
#     levels = int(np.log2(N))
#     twiddles = {}

#     for stage in range(levels):
#         half_size = 2 ** (levels - stage - 1)
#         tw = []
#         for n in range(half_size):
#             k = n * N // (2 * half_size)
#             angle = -2 * np.pi * k / N
#             wr = np.float16(np.cos(angle))
#             wi = np.float16(np.sin(angle))
#             tw.append((wr, wi))
#         twiddles[stage] = tw[::-1]  # reverse index to match 'temp → i'
#     return twiddles



# def fft_serial_with_twiddle_fp16(x, twiddles_fp16):
#     """
#     Serial FFT using precomputed twiddle factors (DIF, radix-2), all in float16.

#     Modified:
#         - Twiddle 곱한 결과를 앞쪽 인덱스 (i)에 저장함
#         - 결과는 원래 DIF 구조와 동일하게 출력되도록 후처리 포함

#     Inputs:
#         x: list of (real, imag) tuples (float or float16)
#         twiddles_fp16: dict[stage] = list of (real, imag) twiddles as float16

#     Returns:
#         output: list of (real, imag) float16 tuples (bit-reversed)
#         stage_value: list of intermediate stage outputs (each is list of [real, imag])
#     """
#     x_real = np.array([np.float16(a[0]) for a in x], dtype=np.float16)
#     x_imag = np.array([np.float16(a[1]) for a in x], dtype=np.float16)
#     N = len(x_real)
#     levels = int(np.log2(N))

#     reordered_real = np.copy(x_real)
#     reordered_imag = np.copy(x_imag)

#     stage_value = []

#     for stage in range(levels):
#         tmp_value = []
#         half_size = 2 ** (levels - stage - 1)
#         step = 2 * half_size
#         stage_twiddles = twiddles_fp16[stage]  # 순차적으로 사용
#         # stage_twiddles = twiddles_fp16[stage][::-1]  # 리스트 역순

#         for k in range(0, N, step):
#             for n in range(half_size):
#                 i = k + n
#                 j = i + half_size

#                 a_real = reordered_real[i]
#                 a_imag = reordered_imag[i]
#                 b_real = reordered_real[j]
#                 b_imag = reordered_imag[j]

#                 # A + B
#                 t_real = np.float16(a_real + b_real)
#                 t_imag = np.float16(a_imag + b_imag)

#                 # A - B * twiddle
#                 d_real = np.float16(a_real - b_real)
#                 d_imag = np.float16(a_imag - b_imag)

#                 wr, wi = stage_twiddles[n]
#                 temp_real = np.float16(wr * d_real - wi * d_imag)
#                 temp_imag = np.float16(wr * d_imag + wi * d_real)

#                 # 위치 바꿔 저장: temp → i, sum → j
#                 reordered_real[i] = temp_real
#                 reordered_imag[i] = temp_imag
#                 reordered_real[j] = t_real
#                 reordered_imag[j] = t_imag

#         # 저장
#         for idx in range(N):
#             tmp_value.append([reordered_real[idx], reordered_imag[idx]])
#         stage_value.append(tmp_value.copy())

#     # 🧩 마지막 스테이지 후 위치 swap: (i, j)로 저장된 것 원래대로 돌림
#     # 이걸 안하면 출력 순서가 바뀌게 됨
#     half_size = 1
#     step = 2
#     for k in range(0, N, step):
#         i = k
#         j = k + 1
#         reordered_real[i], reordered_real[j] = reordered_real[j], reordered_real[i]
#         reordered_imag[i], reordered_imag[j] = reordered_imag[j], reordered_imag[i]

#     # Bit-reversal
#     def bit_reverse(n, bits):
#         rev = 0
#         for _ in range(bits):
#             rev = (rev << 1) | (n & 1)
#             n >>= 1
#         return rev

#     final_real = np.zeros(N, dtype=np.float16)
#     final_imag = np.zeros(N, dtype=np.float16)
#     for i in range(N):
#         j = bit_reverse(i, levels)
#         final_real[j] = reordered_real[i]
#         final_imag[j] = reordered_imag[i]

#     output = [[final_real[i], final_imag[i]] for i in range(N)]

#     return output, stage_value

# def fft_serial_with_twiddle_fp16(x, twiddles_fp16):
#     """
#     Radix-2 DIF FFT using precomputed twiddle factors.
#     Stores twiddle result in i, sum in j. Output matches standard DIF FFT.

#     Inputs:
#         x: list of (real, imag) float16 values
#         twiddles_fp16: dict[stage] = list of (wr, wi)

#     Returns:
#         output: bit-reversed float16 FFT result
#         stage_value: per-stage intermediate values
#     """
#     x_real = np.array([np.float16(a[0]) for a in x], dtype=np.float16)
#     x_imag = np.array([np.float16(a[1]) for a in x], dtype=np.float16)
#     N = len(x_real)
#     levels = int(np.log2(N))

#     reordered_real = np.copy(x_real)
#     reordered_imag = np.copy(x_imag)
#     stage_value = []

#     for stage in range(levels):
#         half_size = 2 ** (levels - stage - 1)
#         step = 2 * half_size
#         stage_twiddles = twiddles_fp16[stage]

#         for k in range(0, N, step):
#             for n in range(half_size):
#                 i = k + n
#                 j = i + half_size

#                 a_real = reordered_real[i]
#                 a_imag = reordered_imag[i]
#                 b_real = reordered_real[j]
#                 b_imag = reordered_imag[j]

#                 # A + B
#                 t_real = np.float16(a_real + b_real)
#                 t_imag = np.float16(a_imag + b_imag)

#                 # A - B
#                 d_real = np.float16(a_real - b_real)
#                 d_imag = np.float16(a_imag - b_imag)

#                 # Twiddle × (A - B)
#                 wr, wi = stage_twiddles[n]
#                 temp_real = np.float16(wr * d_real - wi * d_imag)
#                 temp_imag = np.float16(wr * d_imag + wi * d_real)

#                 # 저장 순서 바꿈
#                 reordered_real[i] = temp_real
#                 reordered_imag[i] = temp_imag
#                 reordered_real[j] = t_real
#                 reordered_imag[j] = t_imag

#         stage_value.append([[reordered_real[i], reordered_imag[i]] for i in range(N)])

#     # 마지막 stage에서 i-j 위치 다시 원위치 swap
#     half_size = 1
#     step = 2
#     for k in range(0, N, step):
#         i = k
#         j = k + 1
#         reordered_real[i], reordered_real[j] = reordered_real[j], reordered_real[i]
#         reordered_imag[i], reordered_imag[j] = reordered_imag[j], reordered_imag[i]

#     # Bit-reversal
#     def bit_reverse(n, bits):
#         rev = 0
#         for _ in range(bits):
#             rev = (rev << 1) | (n & 1)
#             n >>= 1
#         return rev

#     final_real = np.zeros(N, dtype=np.float16)
#     final_imag = np.zeros(N, dtype=np.float16)
#     for i in range(N):
#         j = bit_reverse(i, levels)
#         final_real[j] = reordered_real[i]
#         final_imag[j] = reordered_imag[i]

#     output = [[final_real[i], final_imag[i]] for i in range(N)]
#     return output, stage_value


# Store all necessary info

In [31]:

N = 256

x = generate_fp16(size=N, complex=True)
twiddles_fp16 = generate_twiddles_fp16(N)
out, stages = fft_serial_with_twiddle_fp16(x, twiddles_fp16)

print(x[0])

store_binary(x, f'../PE/DATA/fft_0_input.txt')

for i in range(int(np.log2(N))):
    store_binary(twiddles_fp16[i], f'../PE/DATA/fft_{i}_weight.txt')
    store_binary(stages[i], f'../PE/DATA/fft_{i}_output.txt')
    store_binary(stages[i], f'../PE/DATA/fft_{i+1}_input.txt')


[0.9185, -0.925]


# Reference

In [15]:

N = 16
x = np.random.randn(N) + 1j * np.random.randn(N)

oring_x = x.copy()

x_real = np.float16(x.real)
x_imag = np.float16(x.imag)
x = []

for i in range(N):
    x.append([x_real[i], x_imag[i]])


twiddles_fp16 = generate_twiddles_fp16(N)
# twiddles_diff = generate_twiddle_factors_fp16_dif(N)

# out, stages = fft_serial_with_twiddle_fp16(x, twiddles_diff)
out, stages = fft_serial_with_twiddle_fp16(x, twiddles_fp16)
out2, stages2 = fft_1(x_real, x_imag, twiddles_fp16)
ref = fft(oring_x)

for i in range(10):
    print(f"{out[i][0]:.5f} {out[i][1]:.5f}", out2[i][0], out2[i][1], ref[i])

-1.43359 -0.29443 -1.434 -0.2944 (-1.4341952656395733-0.29330531135201454j)
-4.41406 -1.56250 -4.41 -1.564 (-4.4106077617664425-1.5634441244365656j)
5.50781 3.23438 5.508 3.234 (5.508855307414229+3.235492010326293j)
1.93750 6.05469 1.942 6.055 (1.9364580415021742+6.056341425119294j)
-2.52539 -0.07715 -2.525 -0.07715 (-2.524593885987997-0.07570594502612948j)
-2.60938 -7.92969 -2.61 -7.93 (-2.6122585724027623-7.928558706625248j)
-2.04492 2.91992 -2.045 2.92 (-2.0448618854367355+2.921242994444687j)
-0.20361 4.14844 -0.1992 4.145 (-0.19995302780032287+4.143786930244717j)
1.44141 0.35010 1.441 0.35 (1.4395795320160687+0.35022762790322304j)
-1.14258 4.84375 -1.144 4.844 (-1.1450041345226416+4.845601273097778j)


In [3]:
def fft_dit_with_twiddle_fp16(x, twiddles_fp16):
    """
    Serial FFT using precomputed twiddle factors (DIT, radix-2), all in float16.

    Inputs:
        x: list of (real, imag) tuples (float or float16)
        twiddles_fp16: dict[stage] = list of (real, imag) twiddles as float16

    Returns:
        output: list of (real, imag) float16 tuples (natural order)
        stage_value: list of intermediate stage outputs (each is list of [real, imag])
    """
    N = len(x)
    levels = int(np.log2(N))

    def bit_reverse(n, bits):
        rev = 0
        for _ in range(bits):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev

    # Bit-reversed input
    x_rev = [x[bit_reverse(i, levels)] for i in range(N)]
    x_real = np.array([np.float16(a[0]) for a in x_rev], dtype=np.float16)
    x_imag = np.array([np.float16(a[1]) for a in x_rev], dtype=np.float16)

    stage_value = []

    # DIT Butterfly computation
    for stage in range(levels):
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = twiddles_fp16[stage]

        for k in range(0, N, step):
            for j in range(half_size):
                i = k + j
                l = i + half_size

                a_real = x_real[i]
                a_imag = x_imag[i]
                b_real = x_real[l]
                b_imag = x_imag[l]

                wr, wi = stage_twiddles[j]
                # B * W
                tmp_real = np.float16(wr * b_real - wi * b_imag)
                tmp_imag = np.float16(wr * b_imag + wi * b_real)

                # Butterfly
                x_real[i] = np.float16(a_real + tmp_real)
                x_imag[i] = np.float16(a_imag + tmp_imag)
                x_real[l] = np.float16(a_real - tmp_real)
                x_imag[l] = np.float16(a_imag - tmp_imag)

        # Save per-stage result
        tmp_value = []
        for idx in range(N):
            tmp_value.append([x_real[idx], x_imag[idx]])
        stage_value.append(tmp_value.copy())

    # Output as list of (real, imag)
    output = [[x_real[i], x_imag[i]] for i in range(N)]

    return output, stage_value


In [4]:
import numpy as np

def generate_twiddle_factors_fp16_dit(N):
    """
    Generate twiddle factors for radix-2 DIT FFT.
    Returns dict[stage] = list of (wr, wi)
    """
    levels = int(np.log2(N))
    twiddles = {}
    for stage in range(levels):
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = []
        for n in range(half_size):
            k = n * N // step
            angle = -2 * np.pi * k / N
            wr = np.float16(np.cos(angle))
            wi = np.float16(np.sin(angle))
            stage_twiddles.append((wr, wi))
        twiddles[stage] = stage_twiddles
    return twiddles

def fft_dit_with_twiddle_fp16(x, twiddles_fp16):
    N = len(x)
    levels = int(np.log2(N))

    def bit_reverse(n, bits):
        rev = 0
        for _ in range(bits):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev

    # Bit-reversed input
    x_rev = [x[bit_reverse(i, levels)] for i in range(N)]
    x_real = np.array([np.float16(a[0]) for a in x_rev], dtype=np.float16)
    x_imag = np.array([np.float16(a[1]) for a in x_rev], dtype=np.float16)

    stage_value = []

    for stage in range(levels):
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = twiddles_fp16[stage]

        for k in range(0, N, step):
            for j in range(half_size):
                i = k + j
                l = i + half_size

                a_real = x_real[i]
                a_imag = x_imag[i]
                b_real = x_real[l]
                b_imag = x_imag[l]

                wr, wi = stage_twiddles[j]
                tmp_real = np.float16(wr * b_real - wi * b_imag)
                tmp_imag = np.float16(wr * b_imag + wi * b_real)

                x_real[i] = np.float16(a_real + tmp_real)
                x_imag[i] = np.float16(a_imag + tmp_imag)
                x_real[l] = np.float16(a_real - tmp_real)
                x_imag[l] = np.float16(a_imag - tmp_imag)

        tmp_value = []
        for idx in range(N):
            tmp_value.append([x_real[idx], x_imag[idx]])
        stage_value.append(tmp_value.copy())

    output = [[x_real[i], x_imag[i]] for i in range(N)]
    return output, stage_value

# Set input size
N = 256
np.random.seed(42)
x = np.random.uniform(-1, 1, size=(N, 2)).astype(np.float16).tolist()

# Generate DIT twiddles and run FFT
twiddles_dit = generate_twiddle_factors_fp16_dit(N)
output_dit, stages_dit = fft_dit_with_twiddle_fp16(x, twiddles_dit)

# Reference with numpy
x_np = np.array([complex(a[0], a[1]) for a in x], dtype=np.complex64)
ref = np.fft.fft(x_np)
ref_fp16 = np.array([[np.float16(c.real), np.float16(c.imag)] for c in ref])

# Compare
for i in range(10):
    print(f"{output_dit[i][0]:.5f} {output_dit[i][1]:.5f}", ref_fp16[i][0], ref_fp16[i][1])


-4.65234 6.50000 -4.656 6.508
-20.26562 5.71484 -20.25 5.703
-7.09375 8.78906 -7.086 8.79
-1.37891 -8.95312 -1.372 -8.94
8.85156 5.25781 8.85 5.258
1.68359 4.16797 1.686 4.164
-1.67969 1.42090 -1.67 1.423
-8.05469 -6.51953 -8.05 -6.516
7.77344 11.60938 7.777 11.6
3.46484 4.18750 3.48 4.176


# DIT version final

In [3]:
import numpy as np
import random
from FPU import cmul, cadd
from utils import read_binary, binary_to_fp16, store_binary, generate_fp16

# 1. Bit-reversal index generator
def bit_reverse_indices(N):
    levels = int(np.log2(N))
    def reverse(n):
        rev = 0
        for _ in range(levels):
            rev = (rev << 1) | (n & 1)
            n >>= 1
        return rev
    return [reverse(i) for i in range(N)]

# 2. Twiddle factor generator for DIT
def generate_twiddle_factors_fp16_dit(N):
    levels = int(np.log2(N))
    twiddles = {}
    for stage in range(levels):
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = []
        for n in range(half_size):
            k = n * N // step
            angle = -2 * np.pi * k / N
            wr = np.float16(np.cos(angle))
            wi = np.float16(np.sin(angle))
            stage_twiddles.append((wr, wi))
        twiddles[stage] = stage_twiddles
    return twiddles

# 3. DIT FFT without internal bit-reversal
def fft_dit_with_twiddle_fp16_no_bitrev(x, twiddles_fp16):
    N = len(x)
    levels = int(np.log2(N))

    x_real = np.array([np.float16(a[0]) for a in x], dtype=np.float16)
    x_imag = np.array([np.float16(a[1]) for a in x], dtype=np.float16)

    stage_value = []

    for stage in range(levels):
        half_size = 2 ** stage
        step = 2 * half_size
        stage_twiddles = twiddles_fp16[stage]

        # print(half_size, step, stage_twiddles)

        for k in range(0, N, step):
            for j in range(half_size):
                i = k + j
                l = i + half_size

                a_real = x_real[i]
                a_imag = x_imag[i]
                b_real = x_real[l]
                b_imag = x_imag[l]

                wr, wi = stage_twiddles[j]
                tmp_real = np.float16(wr * b_real - wi * b_imag)
                tmp_imag = np.float16(wr * b_imag + wi * b_real)

                x_real[i] = np.float16(a_real + tmp_real)
                x_imag[i] = np.float16(a_imag + tmp_imag)
                x_real[l] = np.float16(a_real - tmp_real)
                x_imag[l] = np.float16(a_imag - tmp_imag)

        stage_value.append([[x_real[i], x_imag[i]] for i in range(N)])

    output = [[x_real[i], x_imag[i]] for i in range(N)]
    return output, stage_value

In [11]:
## Test and compare with reference numpy FFT

N = 256

x = generate_fp16(size=N, complex=True)
# External bit-reversal
indices = bit_reverse_indices(N)
x_bitrev = [x[i] for i in indices]

# Generate twiddles
twiddles_dit = generate_twiddle_factors_fp16_dit(N)

# Run DIT FFT
output_dit, stages_dit = fft_dit_with_twiddle_fp16_no_bitrev(x_bitrev, twiddles_dit)

# Compare to numpy
x_np = np.array([complex(a[0], a[1]) for a in x], dtype=np.complex64)
ref = np.fft.fft(x_np)
ref_fp16 = np.array([[np.float16(c.real), np.float16(c.imag)] for c in ref])

# Compare
for i in range(10):
    print(f"{output_dit[i][0]:.5f} {output_dit[i][1]:.5f}", ref_fp16[i][0], ref_fp16[i][1])

store_binary(x_bitrev, f'../PE/DATA/fft_0_input.txt')

for i in range(int(np.log2(N))):
    store_binary(twiddles_dit[i], f'../PE/DATA/fft_{i}_weight.txt')
    store_binary(stages_dit[i], f'../PE/DATA/fft_{i}_output.txt')
    if(i == 0):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=4)
    if(i == 1):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=11)
    if(i == 2):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=20)
    if(i == 3):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=33)
    if(i == 4):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=54)
    if(i == 5):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=91)
    if(i == 6):
        store_binary(stages_dit[i], f'../PE/DATA/fft_{i+1}_input.txt', zero_padding=160)

127.12500 124.50000 127.1 124.56
-4.82031 -11.82812 -4.82 -11.84
-5.67188 -12.17969 -5.676 -12.18
-4.95703 -15.37500 -4.95 -15.375
-26.40625 -5.83984 -26.39 -5.848
17.01562 -7.63672 17.0 -7.633
-5.51172 -31.26562 -5.523 -31.25
-3.81055 -1.58203 -3.812 -1.59
-8.53906 2.58789 -8.54 2.574
-24.01562 -23.59375 -24.02 -23.6


In [7]:
for idx, i in enumerate(stages_dit[5]):
    print(idx, i)

0 [29.1, 27.81]
1 [5.38, -6.652]
2 [-2.414, -9.914]
3 [0.3652, 0.06445]
4 [-6.625, -3.576]
5 [9.766, -6.008]
6 [0.2324, -6.734]
7 [-12.805, 4.38]
8 [8.484, 2.258]
9 [-12.25, -2.504]
10 [1.93, -7.25]
11 [-0.789, 3.162]
12 [-10.87, -4.695]
13 [6.527, -5.617]
14 [-4.812, -2.059]
15 [-11.03, 1.099]
16 [8.19, -2.922]
17 [6.773, 4.086]
18 [-6.457, -12.695]
19 [-2.75, 4.676]
20 [8.6, 10.59]
21 [9.71, 3.428]
22 [-3.469, -11.9]
23 [4.22, -9.86]
24 [3.66, -2.965]
25 [-3.06, -0.671]
26 [-5.305, -3.982]
27 [1.621, -0.3867]
28 [-9.805, -9.055]
29 [-6.145, -8.3]
30 [-10.3, 8.516]
31 [6.094, 1.697]
32 [-7.523, 4.617]
33 [9.83, -8.07]
34 [-0.596, -7.1]
35 [5.516, 7.53]
36 [8.984, -1.229]
37 [3.0, 2.928]
38 [6.57, 4.68]
39 [6.367, 1.457]
40 [0.8438, -2.672]
41 [8.875, -5.285]
42 [12.18, 2.963]
43 [1.609, 8.22]
44 [-2.898, -10.9]
45 [7.254, 1.527]
46 [-9.99, 6.37]
47 [1.055, -0.4307]
48 [7.387, 4.656]
49 [15.555, 7.67]
50 [0.4492, -5.914]
51 [4.934, 0.668]
52 [-7.04, -14.92]
53 [-3.38, -4.43]
54 [-4.61,

In [3]:
twiddles_dit[0]

[(1.0, -0.0)]